In [43]:
# Auto reload modules
%load_ext autoreload
%autoreload all

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
import os
from typing import List
import numpy as np
from numpy.linalg import LinAlgError
import pandas as pd
from pprint import pprint
import tqdm

from weac_2.analysis import Analyzer
from weac_2.core.system_model import SystemModel
from weac_2.components import ModelInput, Segment, ScenarioConfig, WeakLayer, Layer
from weac_2.utils.snowpilot_parser import SnowPilotParser, convert_to_mm, convert_to_deg


---
# Extract All the PST files

In [45]:

# Process multiple files
file_paths = []
for directory in os.listdir("data/snowpits"):
    for file in os.listdir(f"data/snowpits/{directory}"):
        if file.endswith(".xml"):
            file_paths.append(f"data/snowpits/{directory}/{file}")

pst_paths: List[str] = []
pst_parsers: List[SnowPilotParser] = []
amount_of_psts = 0

for file_path in file_paths:
    snowpilot_parser = SnowPilotParser(file_path)
    if len(snowpilot_parser.snowpit.stability_tests.PST) > 0:
        pst_paths.append(file_path)
        pst_parsers.append(snowpilot_parser)
        amount_of_psts += len(snowpilot_parser.snowpit.stability_tests.PST)

print(f"\nFound {len(pst_paths)} files with PST tests")
print(f"Found {amount_of_psts} PST tests")


Found 3102 files with PST tests
Found 3719 PST tests


---
# Run WEAC with Geldsetzer & Density Parameterization for WeakLayer


In [46]:
# Extract data from all PST files
error_paths = {}
error_values = {}
failed_to_extract_layers = 0
overall_excluded_psts = 0
cut_length_exceeds_column_length = 0
slope_angle_is_None = 0
failed_to_extract_weak_layer = 0

data_rows = []
for i, (file_path, parser) in tqdm.tqdm(
    enumerate(zip(pst_paths, pst_parsers)), total=len(pst_paths)
):
    try:
        if parser.snowpit.core_info.location.slope_angle is None:
            phi = 0.0
        else:
            phi = (
                parser.snowpit.core_info.location.slope_angle[0]
                * convert_to_deg[parser.snowpit.core_info.location.slope_angle[1]]
            )
        try:
            layers, density_method = parser.extract_layers()
        except Exception as e:
            failed_to_extract_layers += len(parser.snowpit.stability_tests.PST)
            raise e
        for pst_id, pst in enumerate(parser.snowpit.stability_tests.PST):
            try:
                if pst.cut_length[0] >= pst.column_length[0]:
                    cut_length_exceeds_column_length += 1
                    raise ValueError(
                        "Cut length is equal or greater than column length"
                    )
                try:
                    weak_layer, layers_above = (
                        parser.extract_weak_layer_and_layers_above(
                            pst.depth_top[0] * convert_to_mm[pst.depth_top[1]], layers
                        )
                    )
                except Exception as e:
                    failed_to_extract_weak_layer += 1
                    raise e
                cut_length = pst.cut_length[0] * convert_to_mm[pst.cut_length[1]]
                column_length = (
                    pst.column_length[0] * convert_to_mm[pst.column_length[1]]
                )
                segments = [
                    Segment(length=cut_length, has_foundation=False, m=0.0),
                    Segment(
                        length=column_length - cut_length,
                        has_foundation=True,
                        m=0.0,
                    ),
                ]
                scenario_config = ScenarioConfig(system_type="-vpst", phi=phi)
                model_input = ModelInput(
                    weak_layer=weak_layer,
                    layers=layers_above,
                    scenario_config=scenario_config,
                    segments=segments,
                )
                pst_system = SystemModel(model_input=model_input)
                pst_analyzer = Analyzer(pst_system)
                G, GIc, GIIc = pst_analyzer.differential_ERR(unit="J/m^2")

                data_rows.append(
                    {
                        "file_path": file_path,
                        "pst_id": pst_id,
                        "column_length": column_length,
                        "cut_length": cut_length,
                        "phi": phi,
                        # Weak Layer properties
                        "rho_wl": weak_layer.rho,
                        "E_wl": weak_layer.E,
                        "HH_wl": weak_layer.hand_hardness,
                        "GT_wl": weak_layer.grain_type,
                        "GS_wl": weak_layer.grain_size,
                        # Simulation results
                        "G": G,
                        "GIc": GIc,
                        "GIIc": GIIc,
                    }
                )
            except Exception as e:
                error_id = f"{i}.{pst_id}"
                error_paths[error_id] = file_path
                error_values[error_id] = e
                overall_excluded_psts += 1

    except Exception as e:
        error_values[str(i)] = e
        error_paths[str(i)] = file_path
        overall_excluded_psts += len(parser.snowpit.stability_tests.PST)

dataframe = pd.DataFrame(data_rows)
# pprint(error_values)
print(f"\nFound {len(pst_paths)} files with PST tests")
print(f"Found {amount_of_psts} PST tests")
print("Length of the dataframe: ", len(dataframe))
print(f"Amount of excluded PSTs: {overall_excluded_psts}")

print(f"\nFailed to extract layers: {failed_to_extract_layers}")
print(f"Failed to extract weak layer: {failed_to_extract_weak_layer}")
print(f"Slope angle is None: {slope_angle_is_None}")
print(f"Cut length exceeds column length: {cut_length_exceeds_column_length}")
print(
    f"Added Failure Types: {failed_to_extract_layers + slope_angle_is_None + cut_length_exceeds_column_length + failed_to_extract_weak_layer}"
)

100%|██████████| 3102/3102 [00:05<00:00, 584.02it/s]


Found 3102 files with PST tests
Found 3719 PST tests
Length of the dataframe:  3338
Amount of excluded PSTs: 381

Failed to extract layers: 87
Failed to extract weak layer: 18
Slope angle is None: 0
Cut length exceeds column length: 276
Added Failure Types: 381


In [47]:
# exclude dataframes where the cut_length is greater than 60% of the column length
if not dataframe.empty:
    dataframe = dataframe[dataframe["cut_length"] < 0.6 * dataframe["column_length"]]
    print("Length of the dataframe after exclusion: ", len(dataframe))
    print(dataframe.head())

# # Save the data to a csv file
dataframe.to_csv("pst_to_GIc.csv", index=False)

Length of the dataframe after exclusion:  2445
                                          file_path  pst_id  column_length  \
0  data/snowpits/2019-2020/snowpits-19985-caaml.xml       0         1000.0   
1  data/snowpits/2019-2020/snowpits-21226-caaml.xml       0          900.0   
2  data/snowpits/2019-2020/snowpits-21226-caaml.xml       1          900.0   
3  data/snowpits/2019-2020/snowpits-25385-caaml.xml       0         1000.0   
6  data/snowpits/2019-2020/snowpits-20222-caaml.xml       0         1000.0   

   cut_length phi  rho_wl       E_wl HH_wl GT_wl  GS_wl         G       GIc  \
0       350.0  14  158.00   2.839257     F    FC    3.0  0.315035  0.311486   
1       330.0  25  125.00   1.012786    4F  SHxr   10.0  0.531139  0.515946   
2       250.0  25  243.25  18.955973   4F+  DHxr    4.0  0.079346  0.078898   
3       500.0  23  162.88   3.245874   4F-  FCxr    1.0  0.995669  0.981382   
6       380.0  22  125.00   1.012786    4F  SHxr    4.0  0.410701  0.410518   

       GI

---
# Run WEAC with Constant WeakLayer

In [50]:
# Calculate with a standard weak layer
# Extract data from all PST files
error_paths = {}
error_values = {}
failed_to_extract_layers = 0
overall_excluded_psts = 0
cut_length_exceeds_column_length = 0
slope_angle_is_None = 0
failed_to_extract_weak_layer = 0

data_rows = []
standard_weak_layer = WeakLayer(rho=125, h=20, E=1.0)
for i, (file_path, parser) in tqdm.tqdm(
    enumerate(zip(pst_paths, pst_parsers)), total=len(pst_paths)
):
    try:
        if parser.snowpit.core_info.location.slope_angle is None:
            phi = 0.0
        else:
            phi = (
                parser.snowpit.core_info.location.slope_angle[0]
                * convert_to_deg[parser.snowpit.core_info.location.slope_angle[1]]
            )
        try:
            layers, density_method = parser.extract_layers()
        except Exception as e:
            failed_to_extract_layers += len(parser.snowpit.stability_tests.PST)
            raise e
        for pst_id, pst in enumerate(parser.snowpit.stability_tests.PST):
            try:
                if pst.cut_length[0] >= pst.column_length[0]:
                    cut_length_exceeds_column_length += 1
                    raise ValueError(
                        "Cut length is equal or greater than column length"
                    )
                try:
                    weak_layer, layers_above = (
                        parser.extract_weak_layer_and_layers_above(
                            pst.depth_top[0] * convert_to_mm[pst.depth_top[1]], layers
                        )
                    )
                except Exception as e:
                    failed_to_extract_weak_layer += 1
                    raise e
                cut_length = pst.cut_length[0] * convert_to_mm[pst.cut_length[1]]
                column_length = (
                    pst.column_length[0] * convert_to_mm[pst.column_length[1]]
                )
                segments = [
                    Segment(length=cut_length, has_foundation=False, m=0.0),
                    Segment(
                        length=column_length - cut_length,
                        has_foundation=True,
                        m=0.0,
                    ),
                ]
                scenario_config = ScenarioConfig(system_type="-vpst", phi=phi)
                model_input = ModelInput(
                    weak_layer=standard_weak_layer,
                    layers=layers_above,
                    scenario_config=scenario_config,
                    segments=segments,
                )
                pst_system = SystemModel(model_input=model_input)
                pst_analyzer = Analyzer(pst_system)
                G, GIc, GIIc = pst_analyzer.differential_ERR(unit="J/m^2")

                data_rows.append(
                    {
                        "file_path": file_path,
                        "pst_id": pst_id,
                        "column_length": column_length,
                        "cut_length": cut_length,
                        "phi": phi,
                        "cut_depth": pst.depth_top[0] * convert_to_mm[pst.depth_top[1]],
                        # Weak Layer properties
                        "rho_wl": weak_layer.rho,
                        "E_wl": weak_layer.E,
                        "HH_wl": weak_layer.hand_hardness,
                        "GT_wl": weak_layer.grain_type,
                        "GS_wl": weak_layer.grain_size,
                        # Simulation results
                        "G": G,
                        "GIc": GIc,
                        "GIIc": GIIc,
                    }
                )
            except Exception as e:
                error_id = f"{i}.{pst_id}"
                error_paths[error_id] = file_path
                error_values[error_id] = e
                overall_excluded_psts += 1

    except Exception as e:
        error_values[str(i)] = e
        error_paths[str(i)] = file_path
        overall_excluded_psts += len(parser.snowpit.stability_tests.PST)

dataframe_const_wl = pd.DataFrame(data_rows)
# pprint(error_values)
print(f"\nFound {len(pst_paths)} files with PST tests")
print(f"Found {amount_of_psts} PST tests")
print("Length of the dataframe: ", len(dataframe_const_wl))
print(f"Amount of excluded PSTs: {overall_excluded_psts}")

print(f"\nFailed to extract layers: {failed_to_extract_layers}")
print(f"Failed to extract weak layer: {failed_to_extract_weak_layer}")
print(f"Slope angle is None: {slope_angle_is_None}")
print(f"Cut length exceeds column length: {cut_length_exceeds_column_length}")
print(
    f"Added Failure Types: {failed_to_extract_layers + slope_angle_is_None + cut_length_exceeds_column_length + failed_to_extract_weak_layer}"
)

100%|██████████| 3102/3102 [00:05<00:00, 576.28it/s]


Found 3102 files with PST tests
Found 3719 PST tests
Length of the dataframe:  3338
Amount of excluded PSTs: 381

Failed to extract layers: 87
Failed to extract weak layer: 18
Slope angle is None: 0
Cut length exceeds column length: 276
Added Failure Types: 381


In [51]:

# exclude dataframes where the cut_length is greater than 60% of the column length
if not dataframe_const_wl.empty:
    dataframe_const_wl = dataframe_const_wl[dataframe_const_wl["cut_length"] < 0.6 * dataframe_const_wl["column_length"]]
    print("Length of the dataframe after exclusion: ", len(dataframe_const_wl))
    print(dataframe_const_wl.head())

# # Save the data to a csv file
dataframe_const_wl.to_csv("pst_to_GIc_with_const_wl.csv", index=False)

# Transform phi to float
dataframe_const_wl["phi"] = dataframe_const_wl["phi"].astype(float)

# Print largest phi row
phi_max = dataframe_const_wl["phi"].max()
print(dataframe_const_wl[dataframe_const_wl["phi"] == phi_max])

# Print largest GIc row
GIc_max = float(dataframe_const_wl["GIc"].max())
print(dataframe_const_wl[dataframe_const_wl["GIc"] == GIc_max])

# Print largest GIIc row
GIIc_max = float(dataframe_const_wl["GIIc"].max())
print(dataframe_const_wl[dataframe_const_wl["GIIc"] == GIIc_max])

Length of the dataframe after exclusion:  2445
                                          file_path  pst_id  column_length  \
0  data/snowpits/2019-2020/snowpits-19985-caaml.xml       0         1000.0   
1  data/snowpits/2019-2020/snowpits-21226-caaml.xml       0          900.0   
2  data/snowpits/2019-2020/snowpits-21226-caaml.xml       1          900.0   
3  data/snowpits/2019-2020/snowpits-25385-caaml.xml       0         1000.0   
6  data/snowpits/2019-2020/snowpits-20222-caaml.xml       0         1000.0   

   cut_length phi  cut_depth  rho_wl       E_wl HH_wl GT_wl  GS_wl         G  \
0       350.0  14      870.0  158.00   2.839257     F    FC    3.0  0.539426   
1       330.0  25      900.0  125.00   1.012786    4F  SHxr   10.0  0.536080   
2       250.0  25     1050.0  243.25  18.955973   4F+  DHxr    4.0  0.368536   
3       500.0  23      800.0  162.88   3.245874   4F-  FCxr    1.0  2.884303   
6       380.0  22      650.0  125.00   1.012786    4F  SHxr    4.0  0.413342   

   